<h2 style="color: #FFD700;">Web Scraping con API de OpenAI y una API local con el modelo OLLAMA</h2>

El  objetivo es codificar un nuevo tipo de navegador web. Dale una URL y responderá con un resumen.

Primero utilizaremos el modelo de OpenAI

In [3]:
import os  # Importa el módulo 'os' para interactuar con el sistema operativo, como acceder a variables de entorno o rutas de archivos.
import requests # Importa la librería 'requests' para hacer solicitudes HTTP a páginas web.
from bs4 import BeautifulSoup # Importa 'BeautifulSoup' de la librería 'bs4' para analizar el contenido HTML.
from IPython.display import Markdown, display # Importa funciones para mostrar contenido con formato Markdown en notebooks como Jupyter.}
from urllib.parse import urljoin  # Importa 'urljoin' para construir URLs absolutas combinando una base y una ruta relativa.
from dotenv import load_dotenv  # Importa 'load_dotenv' para cargar variables de entorno desde un archivo .env.
from openai import OpenAI  # Importa la clase 'OpenAI' para interactuar con la API de OpenAI.

In [4]:
# Cargamos las variables del fichero .env

load_dotenv() # Lee el fichero .env que contiene la key de la api de openai
api_key = os.getenv('OPENAI_API_KEY')

# Verifica la Key

if not api_key:
    print("No se encontró ninguna clave API")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta")
else:
    print("Se encontró la clave API, funciona correctamente")

Se encontró la clave API, funciona correctamente


In [6]:
# Se crea una instancia del objeto openAI y queda disponible para hacer consultas a la API
openai = OpenAI()

In [11]:
# Una clase para representar una página web
# BeautifulSoup: se conecta a una pagina web para scrapear, analizar y detectar cual es el contenido.

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
# Prueba

xat = Website("https://www.xataka.com/")
print(xat.title)
print(xat.text)

Xataka - Tecnología y gadgets, móviles, informática, electrónica
Webedia
Xataka
Menú
Nuevo
ANÁLISIS
XATAKA SELECCIÓN
MÓVILES
CIENCIA
GUÍAS DE COMPRA
MOVILIDAD
MAGNET
WhatsApp
Tiktok
Telegram
Instagram
Youtube
Twitter
Cuanto más se sabe del apagón en España, menos culpable parece la falta de inercia por las energías renovables
La inercia de la red era la correcta y estaba incluso por encima del objetivo establecido por los operadores europeos
Motorola Razr 60 Ultra, análisis: este es el móvil que necesita cualquier amante de los móviles pequeños
Una de las armas más inquietantes de China ya tiene fecha de vuelo: un enorme dron nodriza con 100 drones kamikaze a bordo
El lanzamiento de las NVIDIA RTX 5060 es un cataclismo reputacional: hay acusaciones de amenazas y chantaje a los medios especializados
HOY SE HABLA DE
AEMET
China
Sam Altman
Nintendo Switch 2
Tim Cook
Bill Gates
Elon Musk
Generación Z
iPhone 16
iPhone 17
Lo último
Europa pone contra las cuerdas a Shein: un mes para presenta

<h2 style="color: #FFD700;">Indicaciones/prompts</h2>

Los modelos como GPT4o han sido entrenados para recibir instrucciones de una manera particular.

Esperan recibir:

**Una indicación del sistema** que les indique qué tarea están realizando y qué tono deben usar. Establece el contexto

**Una indicación del usuario**: el inicio de la conversación al que deben responder. El inicio de la conversacion

In [15]:
# Definimos nuestro mensaje de sistema:

system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

In [16]:
# Una función que escribe un mensaje de usuario que solicita resúmenes de sitios web:

def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt

In [17]:
print(user_prompt_for(xat))

Estás viendo un sitio web titulado Xataka - Tecnología y gadgets, móviles, informática, electrónica
El contenido de este sitio web es el siguiente;     proporciona un breve resumen de este sitio web en formato Markdown.     Si incluye noticias, productos o anuncios, resúmelos también.

Webedia
Xataka
Menú
Nuevo
ANÁLISIS
XATAKA SELECCIÓN
MÓVILES
CIENCIA
GUÍAS DE COMPRA
MOVILIDAD
MAGNET
WhatsApp
Tiktok
Telegram
Instagram
Youtube
Twitter
Cuanto más se sabe del apagón en España, menos culpable parece la falta de inercia por las energías renovables
La inercia de la red era la correcta y estaba incluso por encima del objetivo establecido por los operadores europeos
Motorola Razr 60 Ultra, análisis: este es el móvil que necesita cualquier amante de los móviles pequeños
Una de las armas más inquietantes de China ya tiene fecha de vuelo: un enorme dron nodriza con 100 drones kamikaze a bordo
El lanzamiento de las NVIDIA RTX 5060 es un cataclismo reputacional: hay acusaciones de amenazas y chant

<h2 style="color: #FFD700;">Mensajes</h2>

La API de OpenAI espera recibir mensajes en una estructura particular.
Muchas de las otras API comparten esta estructura:

```
[
    {"role": "system", "content": "el mensaje de sistema va aquí"},
    {"role": "user", "content": "el mensaje de usuario va aquí"}
]

In [18]:
# Esta función crea exactamente el formato anterior

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [19]:
messages_for(xat)

[{'role': 'system',
  'content': 'Eres un asistente que analiza el contenido de un sitio web y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. Responder en Markdown.'},
 {'role': 'user',
  'content': 'Estás viendo un sitio web titulado Xataka - Tecnología y gadgets, móviles, informática, electrónica\nEl contenido de este sitio web es el siguiente;     proporciona un breve resumen de este sitio web en formato Markdown.     Si incluye noticias, productos o anuncios, resúmelos también.\n\nWebedia\nXataka\nMenú\nNuevo\nANÁLISIS\nXATAKA SELECCIÓN\nMÓVILES\nCIENCIA\nGUÍAS DE COMPRA\nMOVILIDAD\nMAGNET\nWhatsApp\nTiktok\nTelegram\nInstagram\nYoutube\nTwitter\nCuanto más se sabe del apagón en España, menos culpable parece la falta de inercia por las energías renovables\nLa inercia de la red era la correcta y estaba incluso por encima del objetivo establecido por los operadores europeos\nMotorola Razr 60 Ultra, análisis: este es el móvil que neces

<h2 style="color: #FFD700;">Unimos todo con la API de OpenAI</h2>

In [21]:
# Llamamos a la API de OpenAI.
# Lo que hace esta celda es resumir una URL pero ahora utilizando GPT-4

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://www.xataka.com/")

In [22]:
# Una función para mostrar esto de forma clara en la salida de Jupyter, usando markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://www.xataka.com/")

# Resumen del sitio web Xataka

Xataka es una plataforma dedicada a la tecnología y los gadgets, ofreciendo contenido variado que incluye análisis, noticias, guías de compra y temáticas sobre móviles, ciencia, movilidad y mucho más.

## Contenidos destacados:

1. **Noticias recientes**:
   - **Apagón en España**: Información sobre la inercia de la red eléctrica y el papel de las energías renovables.
   - **Motorola Razr 60 Ultra**: Análisis de un nuevo modelo de móvil dirigido a los amantes de dispositivos compactos.
   - **NVIDIA RTX 5060**: Revelaciones sobre el lanzamiento de nuevos chips y controversias en torno a la compañía.
   - **Desafío en la robótica**: China organiza un torneo de kickboxing con robots humanoides.
   - **Shein en Europa**: La marca debe presentar cambios en sus políticas de descuentos o enfrentar multas.

2. **Temáticas Emergentes**:
   - **Cultura laboral en Japón**: La generación Z enfrenta el "trabajo ampliado" y promueve un equilibrio entre vida personal y profesional.
   - **Renovables y precios de energía**: Análisis sobre el impacto de las energías renovables en los precios eléctricos.
   - **Tendencias en streaming**: La demanda por planes de suscripción con anuncios rápidamente está en aumento.

3. **Xataka Selección**:
   - Ofertas en tecnología, incluyendo auriculares Marshall y tablets de Xiaomi a precios reducidos.

4. **Inteligencia Artificial y Empresas**:
   - **Sam Altman**: Análisis del crecimiento de OpenAI y sus implicaciones.
   - **NVIDIA y los chips**: Planes para lanzar un chip de IA que será más asequible en el mercado chino.

Xataka es una fuente valiosa para entusiastas de la tecnología, ofreciendo un análisis crítico y actualizaciones sobre las tendencias del sector.

<h2 style="color: #FFD700;">Web Scraping mas avanzado</h2>

Durante la implementación del método display_summary("https://openai.com") u otras urls, se puede notar que no funcionaba correctamente debido a que algunos sitios web estan construidos con tecnologías modernas como JavaScript dinámico, lo cual impide la extracción directa de contenido usando solo librerías como requests y BeautifulSoup.

Para resolver este problema, integré una solución utilizando Selenium (también aplicable con Playwright), que permite automatizar un navegador real, renderizar el contenido dinámico y luego extraer la información relevante del sitio.

Esta mejora fue aplicada en la clase Website, permitiendo realizar scraping en sitios con contenido renderizado del lado del cliente.

In [30]:
# Analizar páginas web que están diseñadas utilizando mucho JavaScript
# Descargar el controlador de Chrome (ChromeDriver)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = 'C:\\Users\\Usuario\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [32]:
display_summary("https://openai.com")

Please complete the verification in the browser and press Enter to continue... 


# Resumen del Sitio Web de OpenAI

OpenAI es una organización dedicada a la investigación y desarrollo de inteligencia artificial avanzada. Su sitio web destaca diversos productos, incluyendo ChatGPT y la plataforma API, diseñada para empresas y desarrolladores.

## Principales secciones del sitio

- **Producción y Productos**:
  - **ChatGPT**: Herramienta de conversación impulsada por IA.
  - **Sora**: Un producto adicional mencionado, aunque no se proporciona un detalle específico.
  - **API Platform**: Plataforma para integrar las capacidades de OpenAI en aplicaciones empresariales.

- **Investigación**:
  - Incluye un índice y resumen de investigaciones actuales y pasadas, con énfasis en la seguridad y privacidad en la IA.
  - Se menciona la introducción de modelos recientes como OpenAI o1, GPT-4o, y el sistema de imagen OpenAI, junto con sus actualizaciones.

- **Noticias y Anuncios**:
  - **Evolving OpenAI’s Structure** y la expansión de liderazgo con Fidji Simo.
  - Anuncios sobre nuevas funciones en API, como la residencia de datos en Asia y la generación de imágenes.
  - Actualizaciones sobre colaboraciones y la aplicación de tecnología en varias industrias, como en el deporte.

## Enfoque en la Seguridad

OpenAI hace un fuerte énfasis en la seguridad y la privacidad, asegurando que las aplicaciones de sus tecnologías sean responsables y éticas.

## Público Objetivo

El sitio está orientado tanto a empresas que buscan integrar la IA en sus soluciones como a investigadores y desarrolladores interesados en la evolución de la inteligencia artificial.

## Conclusión

OpenAI se posiciona como un líder en la investigación y desarrollo de IA, ofreciendo herramientas potentes para la comunicación y la automatización, al tiempo que prioriza la seguridad y la ética en el uso de su tecnología.

<h2 style="color: #FFD700;">Ahora utilizamos el modelo local Ollama</h2>

UAhora utilizarmos un modelo de código abierto que se ejecute localmente a través de Ollama en lugar de OpenAI

Se puede utilizar esta técnica para todos los proyectos posteriores si prefiere no utilizar API de pago (closed source).

In [33]:
# Constantes
# Esto con el nuevo paquete ollama ya no es necesario
OLLAMA_API = "http://localhost:11434/api/chat" # /api/chat punto exacto de llamada para conectarme con el modelo
HEADERS = {"Content-Type": "application/json"} # resultado esperado
MODEL = "llama3.2"

In [34]:
# Se crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI
# Content es la estructura interna
messages = [
    {"role": "user", "content": "Describe algunas de las aplicaciones comerciales de la IA generativa."}
]

In [36]:
# Objeto tipo JSON
payload = {
        "model": MODEL, # modelo
        "messages": messages, # los mensajes del diccionario
        "stream": False #que no transmita los resultados
    }

In [37]:
#Web request, esta es la manera antigua de generarla con OLLAMA, ahora hicieron una forma nueva
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

La inteligencia artificial (IA) generativa es un campo que combina técnicas de aprendizaje automático y procesamiento del lenguaje natural para crear contenido, imágenes, audio y vídeo a partir de patrones y estructuras conocidas.

Algunas de las aplicaciones comerciales más destacadas de la IA generativa son:

1. **Marketing digital**: La IA generativa se utiliza en marketing digital para crear contenido personalizado para los clientes, como anuncios publicitarios, correos electrónicos, artículos y videos. Esto permite a las empresas ofrecer experiencias de marca más personalizadas y efectivas.
2. **Diseno gráfico y visualización**: La IA generativa se utiliza en la creación de diseños gráficos y visualizaciones para productos, paquetes y marketing. Esto incluye la creación de imágenes de producto, logos y materiales de marketing.
3. **Audiovisual**: La IA generativa se utiliza en la creación de contenido audiovisual, como música, efectos sonoros y videos de animación. Esto permite a 

<h2 style="color: #FFD700;">Paquete ollama</h2>

Ahora haremos lo mismo, pero utilizando el paquete de Python ollama en lugar de una llamada HTTP directa.

En esencia, se realiza la misma llamada que se indicó anteriormente al servidor ollama que se ejecuta en localhost:11434

In [38]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

La Inteligencia Artificial (IA) generativa es un campo en constante evolución que se utiliza para crear arte, diseño y contenido automatizadamente. A continuación, te presento algunas de las aplicaciones comerciales más destacadas de la IA generativa:

1. **Diseño gráfico**: La IA generativa puede crear diseños visuales como logotipos, iconos, carteleras y materiales de marketing. Las herramientas como Midjourney y Deep Dream Generator permiten a los usuarios crear diseños personalizados con una gran velocidad y eficiencia.
2. **Musica y audio**: La IA generativa se utiliza para crear música original y audio en vivo. Los algoritmos de aprendizaje automático pueden analizar patrones musicales y generar composiciones únicas. Esto puede ser utilizado en la creación de contenido musical para redes sociales, anuncios publicitarios y producciones cinematográficas.
3. **Imágenes y fotos**: La IA generativa puede crear imágenes y fotos realistas utilizando técnicas como el Deep Learning. Esto 